In [ ]:
import pandas as pd
from numpy import *
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import xgboost as xgb
import warnings
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error, f1_score  # 均方误差
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 100)   # 设置显示最大列数
pd.set_option('display.max_rows', 120)    # 设置显示最大行数
# 读入数据
# 定义数据所在路径
TRAIN_PATH = '../output/WTB_ZJ_A/toUser/train/数据1训练集new.csv'
TEST_PATH = '../output/WTB_ZJ_A/toUser/test/数据1测试集new.csv'

# 利用pandas读入csv数据
train_1 = pd.read_csv(TRAIN_PATH)
test_1 = pd.read_csv(TEST_PATH)
# 查看数据shape
TRAIN_2='../output/WTB_ZJ_A/toUser/train/数据2训练集new.csv'
TEST_2 = '../output/WTB_ZJ_A/toUser/test/数据2测试集new.csv'
train_2=pd.read_csv(TRAIN_2)
test_2=pd.read_csv(TEST_2)
!pip install haversine

In [ ]:
from haversine import haversine

train_2['DIS_N-h']=train_2[['latitude','longitude','home_latitude','home_longitude']].apply(lambda x: haversine((x['longitude'],x['latitude']),(x['home_longitude'],x['home_latitude'])),axis = 1)
train_2['DIS_N-h']=train_2['DIS_N-h']*1000
train_2['DIS_N-w']=train_2[['latitude','longitude','work_latitude','work_longitude']].apply(lambda x: haversine((x['longitude'],x['latitude']),(x['work_longitude'],x['work_latitude'])),axis = 1)
train_2['DIS_N-w']=train_2['DIS_N-w']*1000
test_2['DIS_N-h']=test_2[['latitude','longitude','home_latitude','home_longitude']].apply(lambda x: haversine((x['longitude'],x['latitude']),(x['home_longitude'],x['home_latitude'])),axis = 1)
test_2['DIS_N-h']=test_2['DIS_N-h']*1000
test_2['DIS_N-w']=test_2[['latitude','longitude','work_latitude','work_longitude']].apply(lambda x: haversine((x['longitude'],x['latitude']),(x['work_longitude'],x['work_latitude'])),axis = 1)
test_2['DIS_N-w']=test_2['DIS_N-w']*1000
train_2['DIS_h-w']=train_2[['home_latitude','home_longitude','work_latitude','work_longitude']].apply(lambda x: haversine((x['home_longitude'],x['home_latitude']),(x['work_longitude'],x['work_latitude'])),axis = 1)
train_2['DIS_h-w']=train_2['DIS_h-w']*1000
test_2['DIS_h-w']=test_2[['home_latitude','home_longitude','work_latitude','work_longitude']].apply(lambda x: haversine((x['home_longitude'],x['home_latitude']),(x['work_longitude'],x['work_latitude'])),axis = 1)
test_2['DIS_h-w']=test_2['DIS_h-w']*1000

In [ ]:
def travel_freq(view ,mall ,home,work ,during):
    if(view < 500) or (mall < 500) and (during > 1800) and (home > 1000) and (work > 1000):
        tag = 1
        return tag
    else:
        tag = 0
        return tag
train_2['tag'] = train_2.apply(lambda x:travel_freq(x['stay_viewpoint_distance'],x['stay_mall_distance'],x['DIS_N-h'],x['DIS_N-w'],x['stay_duration']),axis =1)
test_2['tag'] = test_2.apply(lambda x:travel_freq(x['stay_viewpoint_distance'],x['stay_mall_distance'],x['DIS_N-h'],x['DIS_N-w'],x['stay_duration']),axis =1)
def inhome(view1,home1,work1,during1,hour):
    if((view1 > 10) and (during1 < 14400) and ((work1 > 100) or (home1 > 100))):
        if hour >8 and hour < 18:
            homeman = 1
            return homeman
    else:
        homeman = 1
        return homeman
train_2['homeman'] = train_2.apply(lambda x:inhome(x['stay_viewpoint_distance'],x['DIS_N-h'],x['DIS_N-w'],x['stay_duration'],x['in_hour_id']),axis =1)
test_2['homeman'] = test_2.apply(lambda x:inhome(x['stay_viewpoint_distance'],x['DIS_N-h'],x['DIS_N-w'],x['stay_duration'],x['in_hour_id']),axis =1)
County_name_dict = {
    '萧山区':  0,
    '上城区':  1,
    '拱墅区':  2,
    '余杭区':  3,
    '临平区':  4,
    '西湖区':  5,
    '富阳区':  6,
    '淳安县':  7,
    '钱塘区':  8,
    '临安区':  9,
    '滨江区':  10,
    '桐庐县':  11,
    '建德市':  12,
}
train_2['county_name'] = train_2['county_name'].map(County_name_dict)
test_2['county_name'] = test_2['county_name'].map(County_name_dict)
dt1 = train_2['p_day'].astype(str).str.split("-",expand=True)
dt1.columns = ["年","月","日"]
train_2 = pd.concat([train_2,dt1],axis=1)
train_2['p_day']=pd.to_datetime(train_2['p_day'])
train_2['day']=train_2['p_day'].dt.day
train_2['week']=train_2['p_day'].dt.weekday
dt2 = test_2['p_day'].astype(str).str.split("-",expand=True)
dt2.columns = ["年","月","日"]
test_2 = pd.concat([test_2,dt2],axis=1)
test_2['p_day']=pd.to_datetime(test_2['p_day'])
test_2['day']=test_2['p_day'].dt.day
test_2['week']=test_2['p_day'].dt.weekday
col_to_drop = ['p_day','city_name','年','月',"日"]
train_2 = train_2.drop(col_to_drop, axis=1)
test_2 = test_2.drop(col_to_drop,axis= 1)
train_2=train_2.drop(['longitude','latitude','home_longitude','home_latitude','work_longitude','work_latitude'],axis=1)
test_2=test_2.drop(['longitude','latitude','home_longitude','home_latitude','work_longitude','work_latitude'],axis=1)
f_feas=['DIS_N-h','DIS_N-w','DIS_h-w']
for f in f_feas:
    train_2[f'stay_duration_mean_{f}'] = train_2.groupby('stay_duration')[f].transform('mean')
    test_2[f'stay_duration_mean_{f}'] = test_2.groupby('stay_duration')[f].transform('mean')
    train_2[f'in_hour_id_to_mean_{f}'] = train_2.groupby('in_hour_id')[f].transform('mean')
    test_2[f'in_hour_id_to_mean_{f}'] = test_2.groupby('in_hour_id')[f].transform('mean')
    train_2[f'county_name_to_mean_{f}'] = train_2.groupby('county_name')[f].transform('mean')
    test_2[f'county_name_to_mean_{f}'] = test_2.groupby('county_name')[f].transform('mean')
    train_2[f'stay_viewpoint_distance_mean_{f}'] = train_2.groupby('stay_viewpoint_distance')[f].transform('mean')
    test_2[f'stay_viewpoint_distance_mean_{f}'] = test_2.groupby('stay_viewpoint_distance')[f].transform('mean')
    train_2[f'stay_mall_distance_mean_{f}'] = train_2.groupby('stay_mall_distance')[f].transform('mean')
    test_2[f'stay_mall_distance_mean_{f}'] = test_2.groupby('stay_mall_distance')[f].transform('mean')
    
    train_2[f'stay_duration_std_{f}'] = train_2.groupby('stay_duration')[f].transform('std')
    test_2[f'stay_duration_std_{f}'] = test_2.groupby('stay_duration')[f].transform('std')
    train_2[f'stay_viewpoint_distance_std_{f}'] = train_2.groupby('stay_viewpoint_distance')[f].transform('std')
    test_2[f'stay_viewpoint_distance_std_{f}'] = test_2.groupby('stay_viewpoint_distance')[f].transform('std')
    train_2[f'stay_mall_distance_std_{f}'] = train_2.groupby('stay_mall_distance')[f].transform('std')
    test_2[f'stay_mall_distance_std_{f}'] = test_2.groupby('stay_mall_distance')[f].transform('std')
train_2[f'stay_duration_mean_tag'] = train_2.groupby('stay_duration')['tag'].transform('mean')
test_2[f'stay_duration_mean_tag'] = test_2.groupby('stay_duration')['tag'].transform('mean')
train_2[f'county_name_to_mean_tag'] = train_2.groupby('county_name')['tag'].transform('mean')
test_2[f'county_name_to_mean_tag'] = test_2.groupby('county_name')['tag'].transform('mean')
train_2[f'in_hour_id_to_mean_tag'] = train_2.groupby('in_hour_id')['tag'].transform('mean')
test_2[f'in_hour_id_to_mean_tag'] = test_2.groupby('in_hour_id')['tag'].transform('mean')
train_2[f'stay_viewpoint_distance_mean_tag'] = train_2.groupby('stay_viewpoint_distance')['tag'].transform('mean')
test_2[f'stay_viewpoint_distance_mean_tag'] = test_2.groupby('stay_viewpoint_distance')['tag'].transform('mean')
train_2[f'stay_mall_distance_mean_tag'] = train_2.groupby('stay_mall_distance')['tag'].transform('mean')
test_2[f'stay_mall_distance_mean_tag'] = test_2.groupby('stay_mall_distance')['tag'].transform('mean')
for f in f_feas:
    train_2[f'day_{f}'] = train_2.groupby('day')[f].transform('mean')
    test_2[f'day_{f}'] = test_2.groupby('day')[f].transform('mean')
for f in f_feas:
    train_2[f'week_{f}'] = train_2.groupby('week')[f].transform('mean')
    test_2[f'week_{f}'] = test_2.groupby('week')[f].transform('mean')
import gc
def c():
    gc.collect()
train_2=train_2.groupby('user_id').sum()
test_2=test_2.groupby('user_id').sum()
data_2new = pd.concat([train_2,test_2],axis=0)
def active(calling_c ,calling_d ,calling_n,called_c ,called_d,called_n,tot_c,tot_d,tot):
    if((called_c > 20)or(calling_c > 20)or(tot_c > 35)) and ((called_n > 15)or(calling_n > 15) or (tot > 30)) and ((called_d > 40)or(calling_d > 40) or (tot_d > 70)):
        act = 1
        return act
    else:
        act = 0
        return act
train_1['act'] = train_1.apply(lambda x:active(x['calling_cnt'],x['calling_dur'],x['calling_num'],x['called_cnt'],x['called_dur'],x['called_num'],x['tot_call_cnt'],x['tot_call_dur'],x['tot_call_num']),axis =1)
test_1['act'] = test_1.apply(lambda x:active(x['calling_cnt'],x['calling_dur'],x['calling_num'],x['called_cnt'],x['called_dur'],x['called_num'],x['tot_call_cnt'],x['tot_call_dur'],x['tot_call_num']),axis =1)
data = pd.concat([train_1,test_1],axis=0)
data = data.reset_index(drop = True)
data.shape

In [ ]:
lst = ['travel_keyword_visit_cnt','travel_keyword_active_days','travel_weixin_visit_cnt','travel_weixin_active_days','travel_app_visit_cnt','travel_app_active_days']   
for i in lst:
    data[i] = data[i].fillna(-1)
def get_feat1(df, lst):
    for feat in lst:
        enc = LabelEncoder()
        df[feat] = df[feat].astype('str')
        df[feat] = enc.fit_transform(df[feat])
    return df
feat_lst = ['fir_imei_brand', 'county_name']
data = get_feat1(data, feat_lst)
#data['travel_weixin_visit_cnt'] = data['travel_weixin_visit_cnt'].fillna(-1)
data.dtypes.head()

In [ ]:
data_new = pd.merge(data,data_2new,on='user_id',how='left')
data_new.head()

In [ ]:
def box_age(age):
    if age < 20:
        a_box = 0
        return a_box
    elif age >= 20 and age<40:
        a_box = 1
        return a_box
    elif age >= 40 and age<60:
        a_box = 2
        return a_box
    elif age >= 60 and age<80:
        a_box = 3
        return a_box
    else:
        a_box= 4
        return a_box
data_new['age'] = data_new.apply(lambda x:box_age(x['age']),axis =1)
lst = ['hotel_keyword_visit_cnt','income_level_id','has_car','travel_keyword_visit_cnt','hotel_keyword_active_days','travel_keyword_active_days','fir_imei_brand','sms_mo_cnt','hotel_weixin_visit_cnt','hotel_call_dur','viewpoint_call_cnt','innet_dur','calling_dur','travel_weixin_active_days','hotel_weixin_active_days']
data_new=data_new.drop(lst,axis=1)
data_new=data_new.drop(['viewpoint_sms_cnt'],axis=1)



In [ ]:
lst2=['map_app_active_days','onlineshop_app_visit_cnt','hotel_sms_cnt','map_app_visit_cnt','video_app_active_days']
data_new=data_new.drop(lst2,axis=1)

In [ ]:
used_feat = [f for f in data_new.columns if f not in ['user_id', 'flag']]
pd.set_option('display.max_rows', None) 
x_train = data_new.drop(['flag','user_id'],axis=1)
data_corr = x_train.corrwith(data_new.flag)
df_corr = pd.DataFrame(columns=['features','corr'])
df_corr['features']=  data_corr.index
df_corr['corr'] = data_corr.values
df_corr=df_corr.sort_values(by='corr',ascending=False)

In [ ]:
# 建模
def lgb_model(train_x, train_y, test_x):
    folds = 5
    seed = 2022
    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)

    train = np.zeros(train_x.shape[0])
    test = np.zeros(test_x.shape[0])

    cv_scores = []

    for i, (train_index, valid_index) in enumerate(kf.split(train_x, train_y)):
        print('************************************ {} ************************************'.format(str(i+1)))
        trn_x, trn_y, val_x, val_y = train_x.iloc[train_index], train_y[train_index], train_x.iloc[valid_index], train_y[valid_index]

        train_matrix = lgb.Dataset(trn_x, label=trn_y)
        valid_matrix = lgb.Dataset(val_x, label=val_y)

        params = {
                    'boosting_type': 'gbdt',
                    'objective': 'binary',
                    'metric': ['auc'],
                    'num_leaves': 2 ** 6,
                    'learning_rate': 0.07,
                    'seed': 2022,
                    'nthread': 28,
                    'n_jobs':4,
                }
        
        model = lgb.train(params, train_matrix, 50000, valid_sets=[train_matrix, valid_matrix],verbose_eval=200,early_stopping_rounds=200)
        val_pred = model.predict(val_x, num_iteration=model.best_iteration)
        test_pred = model.predict(test_x, num_iteration=model.best_iteration)
        val_pred = np.around(val_pred,0).astype(int)
        test_pred = np.around(test_pred,0).astype(int)
        train[valid_index] = val_pred
        test = test_pred / kf.n_splits
#         cv_scores.append(roc_auc_score(val_y, val_pred))
        val_pred = list(map(lambda x: 0 if x<0.17930148354859754 else 1, val_pred))
        cv_scores.append(f1_score(val_y, val_pred))
        print(cv_scores)
    print(mean(cv_scores))
    return test
# 划分数据集,并训练模型，预测结果
train_x = data_new[data_new['flag'].notnull()][used_feat].copy()
train_y = data_new[data_new['flag'].notnull()]['flag']
test_x = data_new[data_new['flag'].isnull()][used_feat].copy()
print(train_x.shape, test_x.shape)
lgb_test = lgb_model(train_x, train_y, test_x)

In [ ]:
# 保存结果
def get_submit(test_data, result, save_path):
    submit = pd.DataFrame()
    submit['user_id'] = test_data['user_id']
    submit['flag'] = [1 if i>0.17930148354859754 else 0 for i in result]
    submit.to_csv(save_path, index=None)
    print('结果已写出！')
get_submit(test_1, lgb_test, 'sub3.csv')